In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler

import tensorflow_datasets as tfds
import tensorflow as tf
from keras import layers, models, optimizers, Input, Model
from keras.models import load_model
import tensorflow_addons as tfa

from malleable_network_1D import GeneticNetwork1D, MalleableLayer1D, TerminalLayer1D
from malleable_network_2D import GeneticNetwork2D, MalleableLayer2D, TerminalLayer2D

from genetic_training import EvolutionStructure, create_starter_population_entry

2025-01-22 17:10:42.134636: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/tensorflow_addons/utils/ensure_tf_install.py:53: UserWa

In [2]:
def plot_metrics_after_training(history, added_title_text=""):
    # ['loss', 'acc', 'f1_score', 'precision', 'recall', 'auc', \
    # 'val_loss', 'val_acc', 'val_f1_score', 'val_precision', 'val_recall', 'val_auc']

    for metric_name in ['loss', 'acc', 'f1_score', 'precision', 'recall', 'auc']:
        training_metric = history.history[metric_name]
        val_metric = history.history[f"val_{metric_name}"]
        epochs = range(1, len(training_metric)+1)

        plt.plot(epochs, training_metric, 'bo', label=f'Training {metric_name}')
        plt.plot(epochs, val_metric, 'b', label=f'Validation {metric_name}')
        plt.title(f'Training and validation {metric_name}{added_title_text}')
        plt.legend()
        plt.show()


def preprocess(sample, image_size, num_classes):
    image = tf.image.resize(sample['image'], image_size) / 255.0  # Normalize
    category = tf.one_hot(sample['label'], depth=num_classes)
    return image, category

def plot_prediction_result(test_image):
    plt.figure(figsize=(15, 5))

    plt.subplot(1, 3, 1)
    plt.imshow(test_image)
    plt.title("Image")
    plt.axis("off")

    plt.show()

In [3]:
BATCH_SIZE=32
USE_2D_DATA = True

In [4]:
if USE_2D_DATA:
    LOSS_STR='categorical_crossentropy'
    ACTIVATION_STR='softmax'

    dataset, info = tfds.load('cifar10', split=['train', 'test'], with_info=True, as_supervised=False)
    OUTPUT_SIZE = 10
    INPUT_SHAPE = (32, 32, 3)
    PATCH_NUM = (4,4)
    DATASET_NAME = 'CIFAR-10'

    # dataset, info = tfds.load('cifar100', split=['train', 'test'], with_info=True, as_supervised=False)
    # OUTPUT_SIZE = 100
    # INPUT_SHAPE = (32, 32)
    # PATCH_NUM = (2,2)
    # DATASET_NAME = 'CIFAR-100'

    # tfds checksum does not pass for this one for some reason
    # dataset, info = tfds.load('caltech101', split=['train', 'test'], with_info=True, as_supervised=False)
    # # OUTPUT_SIZE = 101
    # # INPUT_SHAPE = (128, 128)
    # PATCH_NUM = (8,8)
    # # DATASET_NAME = 'caltech 101'

    # dataset, info = tfds.load('imagenette', split=['train', 'validation'], with_info=True, as_supervised=False)
    # OUTPUT_SIZE = 10
    # INPUT_SHAPE = (160, 160)
    # PATCH_NUM = (10,10)
    # DATASET_NAME = 'tiny imagenet'

    # all_imgnet, info = tfds.load('imagenet2012', split='train', shuffle_files=True, as_supervised=False)
    # dataset = all_imgnet.take(5_000)
    # # INPUT_SHAPE = (128, 128)
    # PATCH_NUM = (8,8)
    # OUTPUT_SIZE=None
    # DATASET_NAME = 'imagenet'


    train_dataset = dataset[0]
    test_dataset = dataset[1]

    # Convert the train dataset to NumPy arrays
    train_images = []
    train_labels = []

    for sample in tfds.as_numpy(train_dataset.map(lambda sample: preprocess(sample, INPUT_SHAPE[:2], OUTPUT_SIZE))):
        train_images.append(sample[0])
        train_labels.append(sample[1])

    train_images = np.array(train_images)
    train_labels = np.array(train_labels)

    # Split the data using train_test_split
    X_train, X_val, y_train, y_val = train_test_split(train_images, train_labels, test_size=0.13, random_state=42)

    # Create TensorFlow datasets
    train_data = tf.data.Dataset.from_tensor_slices((X_train, y_train)).shuffle(1024).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
    val_data = tf.data.Dataset.from_tensor_slices((X_val, y_val)).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

    # Process the test dataset
    test_data = test_dataset.map(lambda sample: preprocess(sample, INPUT_SHAPE[:2], OUTPUT_SIZE)).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

    # Print dataset sizes
    print(f"Number of training batches:   {len(train_data)}")
    print(f"Number of validation batches: {len(val_data)}")

else:
    LOSS_STR='binary_crossentropy'
    ACTIVATION_STR='sigmoid'

    # Load the data
    data = pd.read_csv('titanic-model/train.csv')

    # Fill missing values (this is just an example; you may use other imputation methods)
    data['Age'].fillna(data['Age'].median(), inplace=True)
    # data['Fare'].fillna(data['Fare'].median(), inplace=True)
    # data['Embarked'].fillna(data['Embarked'].mode()[0], inplace=True)

    # One-hot encode categorical variables
    categorical_features = ['Sex', 'Embarked']
    encoder = OneHotEncoder(sparse=False)
    encoded_features = encoder.fit_transform(data[categorical_features])

    # Normalize numerical features
    numerical_features = ['Pclass', 'Age', 'Fare', 'SibSp', 'Parch']
    scaler = StandardScaler()
    normalized_features = scaler.fit_transform(data[numerical_features])

    # Combine features
    X = np.concatenate([encoded_features, normalized_features], axis=1)
    y = data['Survived'].values

    # Split the data
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

    INPUT_SHAPE = X_train.shape[1:]
    OUTPUT_SIZE = 1

2025-01-22 17:10:45.270784: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype string and shape [1]
	 [[{{node Placeholder/_1}}]]
2025-01-22 17:10:45.271212: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_3' with dtype int64 and shape [1]
	 [[{{node Placeholder/_3}}]]


Number of training batches:   1360
Number of validation batches: 204


In [7]:
population = [
    (
    create_starter_population_entry(
        GeneticNetwork2D(input_shape=INPUT_SHAPE, output_features=OUTPUT_SIZE, output_activation_str=ACTIVATION_STR,)
    ) if USE_2D_DATA else
    create_starter_population_entry(
        GeneticNetwork1D(input_shape=INPUT_SHAPE, output_features=OUTPUT_SIZE, output_activation_str=ACTIVATION_STR,)
    )
    ) for _ in range(4)
]
print(INPUT_SHAPE)

evolution = EvolutionStructure(
    population,
    X_train,
    y_train,
    X_val,
    y_val,
    optimizer_str='adam',
    loss_str=LOSS_STR
)

(32, 32, 3)


In [ ]:
EPOCHS=1
ITERATIONS = 18

for i in range(ITERATIONS):
    print(f"\n\n--ITERATION {i} out of {ITERATIONS}--\n\n")
    evolution.iterate_population(train_epochs=EPOCHS)

print("\n   <All done with iterating>")




****************************************************************************************************

<0><0><0><0><0><0><0><0><0><0><0><0><0><0><0><0><0><0><0><0>
network rank None (done with 0 epochs)
network fitness None

Genetic Network structure:
CONVOLUTION BASE
MalleableLayer2D(sequential=True)
FEEDFORWARD
MalleableLayer1D(sequential=True)
Output Layer: Dense(units=10, activation=softmax)


<1><1><1><1><1><1><1><1><1><1><1><1><1><1><1><1><1><1><1><1>
network rank None (done with 0 epochs)
network fitness None

Genetic Network structure:
CONVOLUTION BASE
MalleableLayer2D(sequential=True)
FEEDFORWARD
MalleableLayer1D(sequential=True)
Output Layer: Dense(units=10, activation=softmax)


<2><2><2><2><2><2><2><2><2><2><2><2><2><2><2><2><2><2><2><2>
network rank None (done with 0 epochs)
network fitness None

Genetic Network structure:
CONVOLUTION BASE
MalleableLayer2D(sequential=True)
FEEDFORWARD
MalleableLayer1D(sequential=True)
Output Layer: Dense(units=10, activation=softmax)


<3

In [ ]:
evolution.train_population(epochs=EPOCHS)
evolution.calculate_fitness()
evolution.rank_population()
for d in evolution.population:
    network, epochs_done, rank, fitness, stats = d['network'], d['training_reps'], d['rank'], d['fitness'], d['stats']
    print()
    print("-*"*80 + "-")
    print(f"\nNetwork Ranked #{rank+1}:")
    print(f"With a fitness of {fitness} after {epochs_done} epochs...")
    network.print_structure()


for ele in evolution.all_stats:
    print(ele)
print("\n best fitness stats over iterations:")
for ele in evolution.best_stats:
    print(ele)


1360/1360 [==============================] - 34s 24ms/step - loss: 1.4950 - acc: 0.4800 - precision_38: 0.7358 - recall_38: 0.2172 - auc_38: 0.8701
H/W Tensor("genetic_network2d_39/malleable_layer2d_122/malleable_layer2d_124/cond_1/Maximum:0", shape=(), dtype=int32) | Tensor("genetic_network2d_39/malleable_layer2d_122/malleable_layer2d_124/cond_1/Maximum_1:0", shape=(), dtype=int32)
LPH/W RPH/W Tensor("genetic_network2d_39/malleable_layer2d_122/malleable_layer2d_124/cond_1/sub:0", shape=(), dtype=int32) Tensor("genetic_network2d_39/malleable_layer2d_122/malleable_layer2d_124/cond_1/sub_1:0", shape=(), dtype=int32) | Tensor("genetic_network2d_39/malleable_layer2d_122/malleable_layer2d_124/cond_1/sub_2:0", shape=(), dtype=int32) Tensor("genetic_network2d_39/malleable_layer2d_122/malleable_layer2d_124/cond_1/sub_3:0", shape=(), dtype=int32)
H/W Tensor("genetic_network2d_39/malleable_layer2d_122/malleable_layer2d_124/cond_1/Maximum:0", shape=(), dtype=int32) | Tensor("genetic_network2d_39/

In [6]:
# MalleableLayer2D(sequential=False)
#   MalleableLayer2D(sequential=True)
#     MalleableLayer2D(sequential=True)
#       MalleableLayer2D(sequential=True)
#         MalleableLayer2D(sequential=True)
#         Right: TerminalLayer2D(type=Conv2D, vector_rep=ListWrapper([2, 195, 2, 2, 2]))
#     Right: TerminalLayer2D(type=Conv2D, vector_rep=ListWrapper([1, 144, 1, 5, 2]))
#   MalleableLayer2D(sequential=True)
#     Left: TerminalLayer2D(type=Conv2D, vector_rep=ListWrapper([2, 223, 1, 3, 1]))
#     Right: TerminalLayer2D(type=MaxPooling2D, vector_rep=ListWrapper([0, 113, 2, 5, 0]))
# FEEDFORWARD
# MalleableLayer1D(sequential=False)
#   MalleableLayer1D(sequential=True)

if True:
    model = GeneticNetwork2D(input_shape=INPUT_SHAPE, output_features=OUTPUT_SIZE, output_activation_str='softmax', build=True)
    
    model.malleable_convolution_base = MalleableLayer2D(sequential=False,
        left=MalleableLayer2D(sequential=True,
            left=MalleableLayer2D(sequential=True,
                    left=MalleableLayer2D(sequential=True),
                    right=TerminalLayer2D(vector_rep=[2, 195, 2, 2, 2])
                ),
            right=TerminalLayer2D(vector_rep=[1, 144, 1, 5, 2])
        ),
        right=MalleableLayer2D(sequential=True,
            left=TerminalLayer2D(vector_rep=[2, 223, 1, 3, 1]),
            right=TerminalLayer2D(vector_rep=[0, 113, 2, 5, 0])
        )
    )
    model.malleable_feedforward = MalleableLayer1D(sequential=False,
        left=MalleableLayer1D(sequential=True)
    )
    model.set_unbuilt()
    model.force_rebuild()
    model.compile(
        optimizer='adam',
        loss='categorical_crossentropy',
        metrics=[
            'acc',
            # tfa.metrics.F1Score(num_classes=OUTPUT_SIZE, average='weighted'),
            tf.keras.metrics.Precision(),
            tf.keras.metrics.Recall(),
            tf.keras.metrics.AUC()
        ]
    )

    model.fit(
        X_train,
        y_train,
        epochs=1
    )

    dummy_input = tf.random.normal((1,) + model.orig_input_shape)  # Example dummy input
    print(model(dummy_input))  # Perform a dummy pass to initialize everything

/Users/afalcignombp/Library/Python/3.9/lib/python/site-packages/keras/initializers/initializers.py:120: UserWarning: The initializer GlorotUniform is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(
2025-01-22 17:11:05.169360: W tensorflow/core/common_runtime/type_inference.cc:339] Type inference failed. This indicates an invalid graph that escaped type checking. Error message: INVALID_ARGUMENT: expected compatible input types, but input 1:
type_id: TFT_OPTIONAL
args {
  type_id: TFT_PRODUCT
  args {
    type_id: TFT_TENSOR
    args {
      type_id: TFT_BOOL
    }
  }
}
 is neither a subtype nor a supertype of the combined inputs preceding it:
type_id: TFT_OPTIONAL
args {
  type_id: TFT_PRODUCT
  args {
    type_id: TFT_TENSOR
    args {
      type_id: TFT_INT32
    }
  }
}

1360/1360 [==============================] - 325s 238ms/step - loss: 1.5742 - acc: 0.4609 - precision_4: 0.6438 - recall_4: 0.2602 - auc_4: 0.8582
tf.Tensor(
[[4.8426811e-20 9.9990296e-01 3.3783908e-27 2.3053623e-11 2.7707763e-18
  3.4716661e-23 1.3067616e-05 1.2411413e-13 3.0107389e-13 8.3944411e-05]], shape=(1, 10), dtype=float32)
